In [3]:
import ee
import geemap
import geopandas as gpd

import sys
import os

sys.path.append(os.path.abspath(".."))  # Add the parent directory to Python path
from utils.etl import load_exported_datasets
from utils.etl import get_municipalities


ee.Authenticate()

ee.Initialize(project="ee-ciut")

In [4]:
# Constants
BUCKET_NAME = "ciut-tablero"
GOOGLE_CLOUD_PROJECT = "ee-ciut"

In [5]:
datasets = load_exported_datasets()

2025-04-28 15:03:57,021 - utils.etl - INFO - 
Loading population dataset:
2025-04-28 15:03:57,022 - utils.etl - INFO - Prefix: draft_data/worldpop_2020
2025-04-28 15:04:00,356 - utils.etl - INFO - Found files: ['gs://ciut-tablero/draft_data/worldpop_2020.tif']
2025-04-28 15:04:00,356 - utils.etl - INFO - Using asset_id: gs://ciut-tablero/draft_data/worldpop_2020.tif
2025-04-28 15:04:00,356 - utils.etl - INFO - Successfully loaded population dataset
2025-04-28 15:04:00,357 - utils.etl - INFO - 
Loading impervious dataset:
2025-04-28 15:04:00,357 - utils.etl - INFO - Prefix: draft_data/gisa_2019
2025-04-28 15:04:02,403 - utils.etl - INFO - Found files: ['gs://ciut-tablero/draft_data/gisa_2019.tif']
2025-04-28 15:04:02,404 - utils.etl - INFO - Using asset_id: gs://ciut-tablero/draft_data/gisa_2019.tif
2025-04-28 15:04:02,404 - utils.etl - INFO - Successfully loaded impervious dataset
2025-04-28 15:04:02,405 - utils.etl - INFO - 
Loading wetlands dataset:
2025-04-28 15:04:02,405 - utils.et

In [7]:
municipalities = get_municipalities()
basins_gdf = gpd.read_file(
    "/home/nissim/Documents/dev/ciut-tablero/data/dipsoh_cuencas.geojson"
)
basins_ee = geemap.geopandas_to_ee(basins_gdf)
intersecting_basins = basins_ee.filterBounds(municipalities.geometry())
aoi = intersecting_basins.geometry()

2025-04-28 15:04:23,704 - pyogrio._io - INFO - Created 110 records


In [8]:
gisa = datasets["impervious"]
population = datasets["population"]
wetlands = datasets["wetlands"]
nighttime_lights = datasets["nighttime_lights"]
vulnerability = datasets["vulnerability"]

In [9]:
# Create map and center on AOI
Map = geemap.Map(basemap="CartoDB.Positron")
Map.centerObject(aoi, 9)

Map.addLayer(gisa, {}, "GISA 1972–2019")

In [10]:
Map.addLayer(
    population,
    {
        "bands": ["population"],
        "min": 0.0,
        "max": 50.0,
        "palette": ["24126c", "1fff4f", "d4ff50"],
    },
    "WorldPop Population 100m",
)

In [11]:
ntl_vis = {
    "opacity": 1,
    "bands": ["avg_rad"],
    "min": 1,
    "max": 32,
    "palette": ["584d9f", "9c79c1", "c98cbe", "f2d192", "e2ee82"],
}
Map.addLayer(nighttime_lights, ntl_vis, "NTL VIIRS")

wetlands_vis = {
    "min": 180,
    "max": 187,
    "palette": [
        "#CCCCCC",
        "#0000FF",
        "#006400",
        "#00FF00",
        "#00FFFF",
        "#CC99FF",
        "#556B2F",
        "#FFFF99",
        "#D2B48C",
    ],
}
Map.addLayer(wetlands.mask(wetlands.neq(0)), wetlands_vis, "GWL FCS30 Wetlands")

Map.addLayer(
    intersecting_basins.style(
        **{"color": "black", "fillColor": "00000000", "width": 1}
    ),
    {},
    "Intersecting Basins",
)

Map

Map(center=[-35.23620217745599, -57.97610800815036], controls=(WidgetControl(options=['position', 'transparent…